# BoTorch Tutorials

The following information is summarised from BoTorch website.
https://botorch.org/docs/introduction.html

You can revisit the binder link via:
https://gesis.mybinder.org/binder/v2/gh/Gtarcraz/botorch_test/6142d100ae4e9fb8c2b3a3602ecb11ba446888af


## Introduction to BoTorch

BoTorch is a library for Bayesian Optimisation research built on top of PyTorch. 

Bayesian Optimisation (BayesOpt) is an established technique for sequential optimisation of costly-to-evaluate black box functions. It can be applied to a wide variety of problems, including hyperparameter optimisation for machine learning algorithms, A/B testing, as well as many scientific and engineering problems.

## Why Botorch?

1. Improved Developer Efficiency
--> Modular and easily extensible interface for composing Bayesian Optimisation primitives (include probabilitic mdels, acquisition functions and optimisers.